In [1]:
import pandas as pd

# Load your dataframes
respondenti_df = pd.read_excel('RESPONDENTI.xlsx', sheet_name='Respondents')
lab_results_base_df = pd.read_excel('LAB results base.xlsx')  # Adjust filename
lab_results_follow_up_df = pd.read_excel('LAB results follow up.xlsx')  # Adjust filename

# Show all column names with their exact spelling
print("="*60)
print("BASE LAB COLUMNS:")
print(lab_results_base_df.head(1))
print("\nFIRST ROW OF FOLLOW-UP DATA:")
print(lab_results_follow_up_df.head(1))

BASE LAB COLUMNS:
   No.  ID FAM GRP POZN. KOMPLET SEX CHILD  aVYS   aHM  ...  TV-Perc  TV-Z  \
0    4  K4  R2  VN   NaN     YES   M     N  1.78  75.3  ...      NaN   NaN   

   HM-PERC  HM-Z  BMI-PERC  BMI-Z  HM/TV-PERC  HM/TV-Z  ID_ID  MY_RESPONDENT  
0      NaN   NaN       NaN    NaN         NaN      NaN   K4R2            NaN  

[1 rows x 232 columns]

FIRST ROW OF FOLLOW-UP DATA:
   ID FAM GRP                                           POZN. SEX CHILD  \
0  K4  R2  VN   + přírustek K337 = Manda Albert; ZMĚNA STRAV.   M     N   

    cVYS   cHM  cPAS  cBOKY  ...           cS-VITB12-PREP  cS-VITB12-VYR  \
0  1.783  75.6  80.0  103.0  ...  Vit B12 methylkobalamin     Metabolics   

   cS-VITB12-DAVKA  cS-VITB12-TYDEN  cS-JOD-PREP  cS-JOD-VYR  cS-JOD-DAVKA  \
0          2 kap/M              NaN           NE         NaN           NaN   

   cS-JOD-TYDEN  ID_ID  MY_RESPONDENTS  
0           NaN   K4R2             YES  

[1 rows x 142 columns]


We checked for right name of columns in LAB results base. It shows there is aCRP and I know in LAB results follow up is cCRP

In [2]:
# Use the correct column names from the diagnostic output
BASE_CRP_COL = 'aCRP'  # Replace with actual name from diagnostic
FOLLOWUP_CRP_COL = 'cCRP'  # Replace with actual name from diagnostic

base_crp = lab_results_base_df[['ID_ID', BASE_CRP_COL]].rename(columns={BASE_CRP_COL: 'CRP_base'})
followup_crp = lab_results_follow_up_df[['ID_ID', FOLLOWUP_CRP_COL]].rename(columns={FOLLOWUP_CRP_COL: 'CRP_followup'})

here we see the evidence

In [3]:
import pandas as pd

# 1. Load RESPONDENTI file as the base (left) table
respondenti_df = pd.read_excel('RESPONDENTI.xlsx', sheet_name='Respondents')

# 2. Extract CRP columns from both lab dataframes
# NOTE: Different column names in each file!
base_crp = lab_results_base_df[['ID_ID', 'aCRP']].rename(columns={'aCRP': 'CRP_base'})
followup_crp = lab_results_follow_up_df[['ID_ID', 'cCRP']].rename(columns={'cCRP': 'CRP_followup'})

# 3. Merge with RESPONDENTI
final_df_CRP = respondenti_df.merge(base_crp, on='ID_ID', how='left') \
                             .merge(followup_crp, on='ID_ID', how='left')

# 4. Inspect results
print(f"✅ Final dataframe shape: {final_df_CRP.shape}")
print(f"✅ With CRP_base: {final_df_CRP['CRP_base'].notna().sum()}")
print(f"✅ With CRP_followup: {final_df_CRP['CRP_followup'].notna().sum()}")

# Show rows
print(final_df_CRP.head(81))


✅ Final dataframe shape: (81, 8)
✅ With CRP_base: 80
✅ With CRP_followup: 76
    RESP  FAM  GRP    AGE FEMALE/MALE    ID_ID  CRP_base  CRP_followup
0     K4   R2   VN  31.00           M     K4R2      3.99          3.99
1     K5   R2   VN  30.70           F     K5R2      3.99          3.99
2     K7   R3   VN  33.00           F     K7R3      3.99          3.99
3    K15   R6   VN  38.10           M    K15R6      3.99          3.99
4    K16   R6   VN  29.40           F    K16R6      6.50          3.99
..   ...  ...  ...    ...         ...      ...       ...           ...
76  K291  R85   VN  26.60           M  K291R85      3.99          3.99
77  K292  R85   VN  26.10           F  K292R85      3.99          3.99
78  K301  R88   VN  35.60           M  K301R88      3.99          3.99
79  K302  R88   VN  38.30           F  K302R88      3.99          3.99
80   NaN  NaN  NaN  33.95         NaN      NaN       NaN           NaN

[81 rows x 8 columns]


In [4]:
# For better plain-text alignment (like in terminal)
print(final_df_CRP.shape)
print(final_df_CRP.head(81))

(81, 8)
    RESP  FAM  GRP    AGE FEMALE/MALE    ID_ID  CRP_base  CRP_followup
0     K4   R2   VN  31.00           M     K4R2      3.99          3.99
1     K5   R2   VN  30.70           F     K5R2      3.99          3.99
2     K7   R3   VN  33.00           F     K7R3      3.99          3.99
3    K15   R6   VN  38.10           M    K15R6      3.99          3.99
4    K16   R6   VN  29.40           F    K16R6      6.50          3.99
..   ...  ...  ...    ...         ...      ...       ...           ...
76  K291  R85   VN  26.60           M  K291R85      3.99          3.99
77  K292  R85   VN  26.10           F  K292R85      3.99          3.99
78  K301  R88   VN  35.60           M  K301R88      3.99          3.99
79  K302  R88   VN  38.30           F  K302R88      3.99          3.99
80   NaN  NaN  NaN  33.95         NaN      NaN       NaN           NaN

[81 rows x 8 columns]


We displayed CRP which we need for median

In [5]:
# Remove rows that are completely empty (NaN in every column)
final_df_CRP = final_df_CRP.dropna(how='all')

# Ensure we only keep the first 80 valid rows
final_df_CRP = final_df_CRP.head(80)

# Reset index (0 → 79)
final_df_CRP = final_df_CRP.reset_index(drop=True)

# Shift index to 1 → 80
final_df_CRP.index = final_df_CRP.index + 1

print(final_df_CRP.shape)
print(final_df_CRP.head(80))

(80, 8)
    RESP  FAM GRP   AGE FEMALE/MALE    ID_ID  CRP_base  CRP_followup
1     K4   R2  VN  31.0           M     K4R2      3.99          3.99
2     K5   R2  VN  30.7           F     K5R2      3.99          3.99
3     K7   R3  VN  33.0           F     K7R3      3.99          3.99
4    K15   R6  VN  38.1           M    K15R6      3.99          3.99
5    K16   R6  VN  29.4           F    K16R6      6.50          3.99
..   ...  ...  ..   ...         ...      ...       ...           ...
76  K289  R84  VN  33.7           F  K289R84      3.99          3.99
77  K291  R85  VN  26.6           M  K291R85      3.99          3.99
78  K292  R85  VN  26.1           F  K292R85      3.99          3.99
79  K301  R88  VN  35.6           M  K301R88      3.99          3.99
80  K302  R88  VN  38.3           F  K302R88      3.99          3.99

[80 rows x 8 columns]


Here we managed to make it 80 rows and not 0-80 but 81 rows with the last row with nan

In [6]:
# Find all IDs missing follow-up CRP
missing_ids = final_df_CRP[final_df_CRP['CRP_followup'].isna()]['ID_ID']

# Display as clean list
print("IDs missing follow-up CRP:")
print(missing_ids.tolist())

# Display as DataFrame table
print("\nIDs missing follow-up CRP (as table):")
display(missing_ids.to_frame())

# With full details
print("\nFull details of respondents missing follow-up:")
missing_details = final_df_CRP[final_df_CRP['CRP_followup'].isna()][['ID_ID', 'CRP_base', 'CRP_followup']]
display(missing_details)


IDs missing follow-up CRP:
['K32R11', 'K45R15', 'K46R15', 'K171R52']

IDs missing follow-up CRP (as table):


,ID_ID
11,K32R11
14,K45R15
15,K46R15
51,K171R52



Full details of respondents missing follow-up:


,ID_ID,CRP_base,CRP_followup
11,K32R11,3.99,NaN
14,K45R15,3.99,NaN
15,K46R15,3.99,NaN
51,K171R52,3.99,NaN


here are exposed respondents without CRP in follow up, which is weird.... because theres only two who didnt come for follow up LAB, have to ask HONZA - he said the results might have not came up from blood

In [7]:
# Rows with CRP_base > 9
high_crp_base = final_df_CRP[final_df_CRP['CRP_base'] > 9]
print("Respondents with CRP_base > 9:")
display(high_crp_base[['ID_ID', 'CRP_base']])

# Rows with CRP_followup > 9
high_crp_followup = final_df_CRP[final_df_CRP['CRP_followup'] > 9]
print("\nRespondents with CRP_followup > 9:")
display(high_crp_followup[['ID_ID', 'CRP_followup']])

# Rows with CRP > 9 in either measurement
high_crp_either = final_df_CRP[(final_df_CRP['CRP_base'] > 9) | (final_df_CRP['CRP_followup'] > 9)]
print("\nRespondents with CRP > 9 in either base or follow-up:")
display(high_crp_either[['ID_ID', 'CRP_base', 'CRP_followup']])


Respondents with CRP_base > 9:


,ID_ID,CRP_base
27,K71R23,10.7
45,K119R38,10.8
72,K282R82,14.4



Respondents with CRP_followup > 9:


,ID_ID,CRP_followup
45,K119R38,12.7
60,K217R64,9.6



Respondents with CRP > 9 in either base or follow-up:


,ID_ID,CRP_base,CRP_followup
27,K71R23,10.70,3.99
45,K119R38,10.80,12.70
60,K217R64,3.99,9.60
72,K282R82,14.40,3.99


 should we  take out respondents  with >9? its in the exclusion criteria, so we have two options, take them out or leave and address it in the limitations

# we will put them in the limitations

# So now we do median of CRP_base and CRP_followup

In [8]:
# Display all CRP values
print("CRP_base values:")
print(final_df_CRP['CRP_base'].tolist())

print("\nCRP_followup values:")
print(final_df_CRP['CRP_followup'].tolist())

# Calculate medians
median_crp_base = final_df_CRP['CRP_base'].median()
median_crp_followup = final_df_CRP['CRP_followup'].median()

print(f"\nMedian CRP_base: {median_crp_base}")
print(f"Median CRP_followup: {median_crp_followup}")


CRP_base values:
[3.99, 3.99, 3.99, 3.99, 6.5, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 4.1, 3.99, 3.99, 10.7, 3.99, 3.99, 3.99, 3.99, 3.99, 7.8, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 10.8, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 4.5, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 14.4, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99]

CRP_followup values:
[3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, nan, 3.99, 3.99, nan, nan, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 7.6, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 12.7, 3.99, 3.99, 3.99, 3.99, 3.99, nan, 3.99, 3.99, 3.99, 3.99, 3.99, 4.2, 3.99, 3.99, 9.6, 3.99, 3.99, 3.99, 3.99, 8.4, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99]

Median CRP

heres the median and Honza said this: já bych je tam nechal a napsal do limitací, porovnal jejich hodnoty s průměrem, zda to někam táhlo průměr


In [9]:
# Display all CRP values
print("CRP_base values:")
print(final_df_CRP['CRP_base'].tolist())

print("\nCRP_followup values:")
print(final_df_CRP['CRP_followup'].tolist())

# Calculate averages (means)
mean_crp_base = final_df_CRP['CRP_base'].mean()
mean_crp_followup = final_df_CRP['CRP_followup'].mean()

print(f"\nAverage CRP_base: {mean_crp_base}")
print(f"Average CRP_followup: {mean_crp_followup}")


CRP_base values:
[3.99, 3.99, 3.99, 3.99, 6.5, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 4.1, 3.99, 3.99, 10.7, 3.99, 3.99, 3.99, 3.99, 3.99, 7.8, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 10.8, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 4.5, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 14.4, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99]

CRP_followup values:
[3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, nan, 3.99, 3.99, nan, nan, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 7.6, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 12.7, 3.99, 3.99, 3.99, 3.99, 3.99, nan, 3.99, 3.99, 3.99, 3.99, 3.99, 4.2, 3.99, 3.99, 9.6, 3.99, 3.99, 3.99, 3.99, 8.4, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99, 3.99]

Average CR

its slightly increasing the average of the overall CRP values in the dataset

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=442c3d06-fa27-4eb9-95a5-3c4fa7c81d32' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>